### Inputs

In [25]:
import os
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from tqdm import tqdm
import pickle

print('imported')

imported


### Loading all patients segmented tumors

In [5]:
# constants
base_directory = './Brats_only_lesion_max_ROI/'
train_patient_list = os.listdir(base_directory + 'train/')
test_patient_list = os.listdir(base_directory + 'test/')

best_slice_path = lambda p, m: base_directory + m + '/' + p + '/' + p + '_best_slices/' + p + '_seg.nii.gz'

train_seg_list = []
test_seg_list = []
for p in tqdm(train_patient_list):
    train_seg_list.append(nib.load(best_slice_path(p, 'train')).get_fdata())
for p in tqdm(test_patient_list):
    test_seg_list.append(nib.load(best_slice_path(p, 'test')).get_fdata())

print('loaded')

100%|██████████| 200/200 [00:01<00:00, 107.90it/s]

loaded


In [12]:
train_seg_list = np.array(train_seg_list)
test_seg_list = np.array(test_seg_list)

# unique values (classes) we have
print('classes indexes: {}'.format(np.unique(train_seg_list)))

classes indexes: [0 1 2 3 4]


### Computing IoU all classes combined

In [21]:
def iou_metric(a_seg, b_seg):
    a_seg = a_seg.astype('uint8')
    b_seg = b_seg.astype('uint8')
    intersection = np.logical_and(a_seg, b_seg)
    union = np.logical_or(a_seg, b_seg)
    return np.sum(intersection) / np.sum(union)

train_all_c_iou = np.zeros((len(train_seg_list), len(train_seg_list)))
test_all_c_iou = np.zeros((len(test_seg_list), len(test_seg_list)))

for i in tqdm(range(len(train_seg_list))):
    for j in range(len(train_seg_list)):
        train_all_c_iou[i, j] = iou_metric(train_seg_list[i], train_seg_list[j])

for i in tqdm(range(len(test_seg_list))):
    for j in range(len(test_seg_list)):
        test_all_c_iou[i, j] = iou_metric(test_seg_list[i], test_seg_list[j])

100%|██████████| 200/200 [00:08<00:00, 22.76it/s]


### Printing small part of the IoU matrix
The matrix should be a symetric matrix, with the diagonal elements equal to 1.

In [24]:
train_all_c_iou[:5, :5]

array([[1.        , 0.33917583, 0.67217935, 0.54090389, 0.36979552],
       [0.33917583, 1.        , 0.43521595, 0.41382141, 0.32598853],
       [0.67217935, 0.43521595, 1.        , 0.51456065, 0.40246126],
       [0.54090389, 0.41382141, 0.51456065, 1.        , 0.64262218],
       [0.36979552, 0.32598853, 0.40246126, 0.64262218, 1.        ]])

In [26]:
test_all_c_iou[:5, :5]

array([[1.        , 0.43032622, 0.33347377, 0.45736919, 0.48860799],
       [0.43032622, 1.        , 0.28437877, 0.67574932, 0.3743099 ],
       [0.33347377, 0.28437877, 1.        , 0.27403994, 0.20674117],
       [0.45736919, 0.67574932, 0.27403994, 1.        , 0.329718  ],
       [0.48860799, 0.3743099 , 0.20674117, 0.329718  , 1.        ]])